# CASADI Opti stack rewrite 
Fix all the god damn bugs

Assume we have a discrete-time model:
$$x_{t+1} = f(x_t, u_t)$$
Denote a whole MPC trajectory, with states $x_0,x_1,\dots,x_N$ and controls $u_1,\dots,u_N$ as $\mathbf{x}$ and $\mathbf{u}$.

We have positional constraints:
$$Ax \leq b$$

We have some interval constraints:
$$x_{min} \leq x_t \leq x_{max},\ i=0,\dots,N$$
$$u_{min} \leq u_t \leq u_{max},\ i=1,\dots,N$$

We have multiobjective weighted cost function which is probably quadratic.
    $$J(\mathbf{x}, \mathbf{u})$$

The nonlinear model is:
\begin{gather*}
\begin{bmatrix}
\dot x\\ \dot y\\ \dot v\\ \dot\psi
%\\ \dot\beta
\end{bmatrix} =
\begin{bmatrix}
v\cos(\psi + \beta)
\\
v\sin(\psi + \beta)
\\
a
\\
\frac{v}{l_r}\sin(\beta)
%\\
%\frac{v}{l_f + l_r}\tan(\delta_f) - \frac{v}{l_r}\sin(\beta)
\end{bmatrix}, \quad \beta = \tan^{-1}\Big(\frac{l_r}{l_r + l_f}\tan(\delta_f)\Big)
\end{gather*}


Assume zero-order-hold on the controls. and $t$, $t+1$ are separated by time $\Delta t$. A very easy discretization is $x_{t+1} = x_t + \dot x_t \Delta t$

In [ ]:
# Library dependencies
import casadi            as ca
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib        as mpl

# Local dependencies
from roadrunner_2        import Roadrunner, Segment, OutOfRoadPointsException
from KinematicModel import KinematicBicycle

model = KinematicBicycle("base_vehicle_model.json")

N = 30
step=0.05

# system size
n = 4
m = 2

# the 2 vehicle parameters are model.lr and model.lf

In [ ]:
from road                import iso_double_lane_change as test_road
test_road_width = 5.0*np.ones(np.size(test_road)//2)
DESIRED_SPEED = 10.0
n_test_road_pts,_ = np.shape(test_road)

roadrunner = Roadrunner(road_center = test_road, # Road centerline points
                        road_width  = test_road_width,
                        P           = np.size(test_road_width)-1,
                       start_pct = 0.0, end_pct = 1.0) # Number of points to try curve-fitting at a time
roadrunner.reset()
# start a bit ahead of the first point to have space for looking back on the road
roadrunner.advance(step*20*DESIRED_SPEED)

def desired_speed(k:int):
    return DESIRED_SPEED

xy = np.reshape(roadrunner.evaluate(),(2,))
ic = [xy[0], xy[1], DESIRED_SPEED, float(roadrunner.get_angle())]

In [ ]:
from scipy.integrate import ode
def f(t, z, u):
    x_i, y_i, v_i, psi_i = z[0], z[1], z[2], z[3]
    a_i, delta_f_i = u[0], u[1]
    beta = np.arctan(model.lr/(model.lr+model.lf)*np.tan(delta_f_i))
    return [
        v_i*np.cos(psi_i + beta),
        v_i*np.sin(psi_i + beta),
        a_i,
        v_i/model.lr*np.sin(beta),
    ]
def move_forward(z0, u0, step):

    r = ode(f)

    r.set_initial_value(z0, 0.0).set_f_params(u0)
    return r.integrate(r.t+step)

In [ ]:
def run_mpc_iteration(z0, coeffs):
    # z0: initial state
    # coeffs: cost weights
    opti = ca.Opti()
    z0_body = roadrunner.to_body_frame(np.reshape(z0[0:2],(1,2)), angle=z0[-1], offset=z0[0:2])
    z_prev = np.array([z0_body[0,0], z0_body[0,1],z0[2],0.0])
    u_prev = np.zeros(m)
    z = opti.variable(n*N)
    
    u = opti.variable(m*N)
    
    z_guess = np.zeros(n*N)
    u_guess = np.zeros(m*N)

    jerk_cost = 0.0
    steering_change_cost = 0.0
    attractive_cost = 0.0

    for i in range(N):
        z_i = z[i*n:(i+1)*n]
        u_i = u[i*m:(i+1)*m]
        x_i, y_i, v_i, psi_i = z_i[0], z_i[1], z_i[2], z_i[3]
        a_i, delta_f_i = u_i[0], u_i[1]

        zdot = model.zdot(z_i,u_i)

        jerk_cost += (u_i[0]-u_prev[0])**2
        steering_change_cost += (u_i[1]-u_prev[1])**2
        
        v_des = desired_speed(i+1)
        xy,psi,_ = roadrunner.evaluate((i+1)*step*v_des, full_data=True)
        xy_prev,psi_prev,_ = roadrunner.evaluate((i+1)*step*v_des, full_data=True)
        xy = roadrunner.to_body_frame(np.reshape(xy,(1,2)), angle=z0[-1], offset=z0[0:2])
        psi = psi - z0[-1]
        psi_prev = psi_prev - z0[-1]
        
        z_guess[i*n:(i+1)*n] = np.array([xy[0,0], xy[0,1],v_des, psi])
        u_guess[i*m:(i+1)*m] = [(v_des-desired_speed(i))/step, (psi - psi_prev)/step]
        attractive_cost += \
            (v_i-desired_speed(i))**2 + (psi_i-psi)**2 +\
            (x_i-xy[0,0])**2 + (y_i-xy[0,1])**2
        
        # Add the polygon bounds at step k                
        #bounds, p = roadrunner.bound_x(step,i,desired_speed)

        #for (ub, a, b, c, lb) in bounds:
        #    if a != 0:
        #        opti.subject_to(opti.bounded(lb,x_i*a + y_i*b + c,ub))
        #    else:
        #        opti.subject_to(opti.bounded(lb,y_i*b + c,ub))

        z_lb, z_ub = model.z_bound()
        opti.subject_to(opti.bounded(z_lb, z_i, z_ub))
        u_lb, u_ub = model.u_bound()
        opti.subject_to(opti.bounded(u_lb, u_i, u_ub))
        opti.subject_to(opti.bounded(-1e-10, z_prev + step*zdot - z_i,1e-10))
        z_prev = z_i

    opti.set_initial(z, z_guess)
    opti.minimize(jerk_cost*coeffs[0] + steering_change_cost*coeffs[1] + attractive_cost*coeffs[2])
    opti.solver('ipopt')

    sol = opti.solve()
    z = sol.value(z)
    u = sol.value(u)
    z = np.reshape(z,(np.size(z)//n,n))
    u = np.reshape(u,(np.size(u)//m,m))
    z[:,0:2] = roadrunner.to_world_frame(z[:,0:2], angle=z0[-1], offset=z0[0:2])
    z[:,3] += z0[-1]
    return z,u

In [ ]:
def run_mpc_test(costs):
    roadrunner.reset()
    # start a bit ahead of the first point to have space for looking back on the road
    roadrunner.advance(step*20*DESIRED_SPEED)

    xy = np.reshape(roadrunner.evaluate(),(2,))
    z0 = [xy[0], xy[1], DESIRED_SPEED, float(roadrunner.get_angle())]

    runs = 600
    z_history = np.zeros((runs,n))
    u_history = np.zeros((runs,m))
    r=0
    try:
        for i in range(runs):
            z,u = run_mpc_iteration(z0, costs)

            # stuff that doesn't have to do with control
            #dist = step*z[1,2]

            # save data
            #z0 = z[0,:] # hack: [xy[0], xy[1], float(z[1,2]), float(roadrunner.get_angle())]
            z0 = move_forward(z0, u[0,:], step)
            dist = roadrunner.advance_xy(z0[0:2])
            z_history[i,:] = z[0,:]
            u_history[i,:] = u[0,:]
            r = i
    except OutOfRoadPointsException:
        z_history = z_history[0:r,:]
        u_history = u_history[0:r,:]
    return z_history, u_history


# Test runs: baseline CG

In [ ]:
z_history_01,u_history_01 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 0.1]))

In [ ]:
z_history_05,u_history_05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 0.5]))

In [ ]:
z_history_1,u_history_1 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 1.0]))

In [ ]:
#z_history_5,u_history_5 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 5.0]))

In [ ]:
#z_history_10,u_history_10 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 10.0]))

# Test runs: CG 0.5m towards rear axle

In [ ]:
model.lr -= 0.5 # Distance from CG to rear axle decreases
model.lf += 0.5 # corresponding increase in CG-front axle distance
z_history_01_cg_r05,u_history_01_cg_r05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 0.1]))

In [ ]:
z_history_05_cg_r05,u_history_05_cg_r05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 0.5]))

In [ ]:
z_history_1_cg_r05,u_history_1_cg_r05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 1.0]))

In [ ]:
#z_history_5_cg_r05,u_history_5_cg_r05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 5.0]))

In [ ]:
#z_history_10_cg_r05,u_history_10_cg_r05 = run_mpc_test(np.array([100.0, 10.0*180.0/np.pi, 10.0]))

# Plotting

In [ ]:
p = roadrunner.plot(n_points=100)

p.plot(z_history_1[:,0], z_history_1[:,1], label="1.0")
p.plot(z_history_05[:,0], z_history_05[:,1], label="0.5")
p.plot(z_history_01[:,0], z_history_01[:,1], label="0.1")
p.plot(z_history_1_cg_r05[:,0], z_history_1_cg_r05[:,1], label="1.0 (rear CG)")
p.plot(z_history_05_cg_r05[:,0], z_history_05_cg_r05[:,1], label="0.5 (rear CG)")
p.plot(z_history_01_cg_r05[:,0], z_history_01_cg_r05[:,1], label="0.1 (rear CG)")
p.legend()
plt.show()

In [ ]:
#plt.plot(u_history_10[:,0], label="10.0")
#plt.plot(u_history_10_cg_r05[:,0], label="10.0", linestyle='--')
#plt.plot(u_history_5[:,0], label="5.0")
#plt.plot(u_history_5_cg_r05[:,0], label="5.0", linestyle='--')
plt.plot(u_history_1[:,0], label="1.0")
plt.plot(u_history_1_cg_r05[:,0], label="1.0", linestyle='--')
plt.plot(u_history_05[:,0], label="0.5")
plt.plot(u_history_05_cg_r05[:,0], label="0.5", linestyle='--')
plt.plot(u_history_01[:,0], label="0.1", linestyle='-')
plt.plot(u_history_01_cg_r05[:,0], label="0.1", linestyle='--')
plt.legend()
#plt.ylim([-0.5,1.0])
plt.title("Steering control signal")
plt.show()

In [ ]:
#plt.plot(u_history_10[:,1], label="10.0")
#plt.plot(u_history_10_cg_r05[:,0], label="10.0", linestyle='--')
#plt.plot(u_history_5[:,1], label="5.0")
#plt.plot(u_history_5_cg_r05[:,0], label="5.0", linestyle='--')
plt.plot(u_history_1[:,1], label="1.0")
plt.plot(u_history_1_cg_r05[:,1], label="1.0", linestyle='--')
plt.plot(u_history_05[:,1], label="0.5")
plt.plot(u_history_05_cg_r05[:,1], label="0.5", linestyle='--')
plt.plot(u_history_01[:,1], label="0.1", linestyle='-')
plt.plot(u_history_01_cg_r05[:,1], label="0.1", linestyle='--')
plt.legend()
plt.title("Acceleration control signal")
#plt.ylim([-0.3,0.5])
plt.show()

In [ ]:
#diff = u_history_10[:np.size(u_history_10_cg_r05)//2,0]-u_history_10_cg_r05[:,0]
#plt.plot(diff, label="10.0")
#diff0 = u_history_5[:np.size(u_history_5_cg_r05)//2,0]-u_history_5_cg_r05[:,0]
#plt.plot(diff0, label="5.0")
diff1 = u_history_1[:np.size(u_history_1_cg_r05)//2,0]-u_history_1_cg_r05[:,0]
plt.plot(diff1, label="1.0")
diff2 = u_history_05[:np.size(u_history_05_cg_r05)//2,0]-u_history_05_cg_r05[:,0]
plt.plot(diff2, label="0.5")
diff3 = u_history_01[:,0]-u_history_01_cg_r05[:np.size(u_history_01)//2,0]
plt.plot(diff3, label="0.1")
plt.legend()
plt.ylim(-0.005,0.005)
plt.title("Acceleration difference for CG shift 0.5m back")
plt.show()

In [ ]:
#diff = u_history_10[:np.size(u_history_10_cg_r05)//2,1]-u_history_10_cg_r05[:,1]
#plt.plot(diff, label="10.0")
#diff0 = u_history_5[:np.size(u_history_5_cg_r05)//2,1]-u_history_5_cg_r05[:,1]
#plt.plot(diff0, label="5.0")
diff1 = u_history_1[:np.size(u_history_1_cg_r05)//2,1]-u_history_1_cg_r05[:,1]
plt.plot(diff1, label="1.0")
diff2 = u_history_05[:np.size(u_history_05_cg_r05)//2,1]-u_history_05_cg_r05[:,1]
plt.plot(diff2, label="0.5")
diff3 = u_history_01[:,1]-u_history_01_cg_r05[:np.size(u_history_01)//2,1]
plt.plot(diff3, label="0.1")
plt.legend()
plt.title("Steering difference for CG shift 0.5m back")
plt.show()